In [1]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd


# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib.ticker as ticker
import seaborn as sns
import pickle

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import OneHotEncoder  
from sklearn.preprocessing import OrdinalEncoder

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder  

from sklearn import tree

import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None



In [2]:
df = pd.read_csv("test.csv", index_col=0)

In [3]:
df.head()

,carat,cut,color,clarity,depth,table,x,y,z
id,,,,,,,,,
0,0.32,Ideal,I,SI1,60.5,58.0,4.43,4.49,2.70
1,1.24,Premium,I,SI1,62.9,60.0,6.80,6.74,4.26
2,1.66,Premium,D,SI1,62.0,59.0,7.55,7.60,4.70
3,0.75,Premium,D,SI2,60.6,56.0,5.94,5.90,3.59
4,1.50,Fair,E,SI2,64.8,55.0,7.26,7.15,4.67


In [4]:
le = LabelEncoder()

In [5]:
def label_encoder(df, columnas):
    for col in df[columnas].columns:
        nuevo_nombre = col + "_encoded"
        df[nuevo_nombre] = le.fit_transform(df[col])
    return df

In [6]:
df_encoded = label_encoder(df, ["cut", "color", "clarity"])
df_encoded.head()

,carat,cut,color,clarity,depth,table,x,y,z,cut_encoded,color_encoded,clarity_encoded
id,,,,,,,,,,,,
0,0.32,Ideal,I,SI1,60.5,58.0,4.43,4.49,2.70,2,5,2
1,1.24,Premium,I,SI1,62.9,60.0,6.80,6.74,4.26,3,5,2
2,1.66,Premium,D,SI1,62.0,59.0,7.55,7.60,4.70,3,0,2
3,0.75,Premium,D,SI2,60.6,56.0,5.94,5.90,3.59,3,0,3
4,1.50,Fair,E,SI2,64.8,55.0,7.26,7.15,4.67,0,1,3


In [7]:
df_encoded2 = df_encoded.drop(['cut', 'color', 'clarity'], axis=1)
df_encoded2.head()

,carat,depth,table,x,y,z,cut_encoded,color_encoded,clarity_encoded
id,,,,,,,,,
0,0.32,60.5,58.0,4.43,4.49,2.70,2,5,2
1,1.24,62.9,60.0,6.80,6.74,4.26,3,5,2
2,1.66,62.0,59.0,7.55,7.60,4.70,3,0,2
3,0.75,60.6,56.0,5.94,5.90,3.59,3,0,3
4,1.50,64.8,55.0,7.26,7.15,4.67,0,1,3


In [8]:
# cargamos el mejor modelo
with open('data/gradient.pkl', 'rb') as modelo:
    modelo = pickle.load(modelo)

In [9]:
# estandarización
with open('data/estandarizacion.pkl', 'rb') as estandarizacion:
    estandarizacion = pickle.load(estandarizacion)

In [10]:
scaler = StandardScaler()
scaler.fit(df_encoded2[['carat', 'x', 'y', 'z']])
X_escaladas = scaler.transform(df_encoded2[['carat', 'x', 'y', 'z']])
df_encoded2[['carat', 'x', 'y', 'z']] = X_escaladas
df_encoded2.head()

,carat,depth,table,x,y,z,cut_encoded,color_encoded,clarity_encoded
id,,,,,,,,,
0,-1.023184,60.5,58.0,-1.176305,-1.132481,-1.222407,2,5,2
1,0.941880,62.9,60.0,0.956477,0.906938,1.040012,3,5,2
2,1.838975,62.0,59.0,1.631409,1.686449,1.678130,3,0,2
3,-0.104730,60.6,56.0,0.182556,0.145555,0.068332,3,0,3
4,1.497225,64.8,55.0,1.370435,1.278565,1.634622,0,1,3


In [11]:
# drop 'depth' and 'table' columns
df_encoded2 = df_encoded2.drop(['depth', 'table'], axis=1)

In [12]:
df_encoded2.head()

,carat,x,y,z,cut_encoded,color_encoded,clarity_encoded
id,,,,,,,
0,-1.023184,-1.176305,-1.132481,-1.222407,2,5,2
1,0.941880,0.956477,0.906938,1.040012,3,5,2
2,1.838975,1.631409,1.686449,1.678130,3,0,2
3,-0.104730,0.182556,0.145555,0.068332,3,0,3
4,1.497225,1.370435,1.278565,1.634622,0,1,3


In [13]:
# hacemos la predicción
precios = modelo.predict(df_encoded2)
precios

array([6.07241374, 8.5447025 , 9.50376894, ..., 6.48739231, 8.69700147,
       7.94094451])

In [14]:
df_precios = pd.DataFrame(precios)

In [15]:
df_precios = df_precios.reset_index(drop=False)

In [16]:
df_precios = df_precios.rename(columns={'index': 'id', 0: 'price'})

In [17]:
df_precios.head()

,id,price
0,0,6.072414
1,1,8.544702
2,2,9.503769
3,3,7.823507
4,4,8.882234


In [18]:
df_precios.to_csv('submit3.csv', index=False)